# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""../../../../data/SampleHDS.jso…","""590231070516660344""","""3293357381555058939""",1.5213e19,"""6336778257352062411""","""3251801070682022300""",false
"""../../../../data/SampleHDS.jso…","""5127350357374060186""","""15929051533558420296""",3.0576e17,"""6336778257352062411""","""9119099936621606514""",true
"""../../../../data/SampleHDS.jso…","""5154843661631139420""","""1414253950885129334""",1.6995e19,"""4870554803191293706""","""3251801070682022300""",false
"""../../../../data/SampleHDS.jso…","""5127350357374060186""","""926602016833024994""",4.2534e18,"""14653275675456387468""","""3251801070682022300""",false
"""../../../../data/SampleHDS.jso…","""590231070516660344""","""4257243693303679486""",1.5413e19,"""14653275675456387468""","""8482189321100963290""",true
"""../../../../data/SampleHDS.jso…","""5154843661631139420""","""13427652130560423901""",1.4572e19,"""1753511158716866305""",null,false
"""../../../../data/SampleHDS.jso…","""5154843661631139420""","""3023981248133468654""",1.3508e19,"""14653275675456387468""","""8482189321100963290""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../data/SampleHDS.jso…","""2390761017977285498""","""12671043976799792194""",6.2783e18,"""FirstMortgage30yr""","""2131059863899129525""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""360863519498995981""","""7066326088787140983""",1.0977e19,"""FirstMortgage30yr""","""9395361297554111368""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""18185124376052095408""","""17989415143786243410""",1.5585e19,"""MoneyMarketSavingsAccount""","""2131059863899129525""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""360863519498995981""","""10096314902821668497""",1.2032e19,"""BasicChecking""","""2131059863899129525""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""2390761017977285498""","""11292823218360934173""",1.5302e18,"""BasicChecking""","""5917848879292837966""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""18185124376052095408""","""17319318291509002386""",6.0435e18,"""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…","""18185124376052095408""","""14283561487272133848""",1.5471e19,"""BasicChecking""","""5917848879292837966""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../data/SampleHDS.jso…","""15891830084921918344""","""13373159406313285242""",9.4908e18,"""FirstMortgage30yr""","""5460846189216633036""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""10992494047791807585""","""4161002353993653886""",1.7085e18,"""FirstMortgage30yr""","""11221789231709514754""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""13486074458484535033""","""7322782813504558805""",1.5401e19,"""MoneyMarketSavingsAccount""","""5460846189216633036""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""10992494047791807585""","""13384471648278146955""",4.8247e18,"""BasicChecking""","""5460846189216633036""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""15891830084921918344""","""9734606889455588114""",1.8017e19,"""BasicChecking""","""627229482128077536""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""13486074458484535033""","""5093165538273966743""",1.8802e18,"""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…","""13486074458484535033""","""14726805934744583803""",1.1034e19,"""BasicChecking""","""627229482128077536""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_MaritalStatus=PREDICTOR_1
Customer_City=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""1159077501598933186""","""13280788742385094854""",6.5083e18,"""FirstMortgage30yr""","""8257679839722446659""","""Rejected"""
"""2904334512040206395""","""8627362139859070894""",1.0040e19,"""FirstMortgage30yr""","""7142188521486966819""","""Accepted"""
"""8477430475517396483""","""16509662661545424465""",4.3419e18,"""MoneyMarketSavingsAccount""","""8257679839722446659""","""Rejected"""
"""2904334512040206395""","""10038783944970368367""",1.6495e19,"""BasicChecking""","""8257679839722446659""","""Rejected"""
"""1159077501598933186""","""12179737289967487959""",5.1416e17,"""BasicChecking""","""2023470075883199825""","""Accepted"""
"""8477430475517396483""","""13454641072525935958""",5.8420e18,"""UPlusFinPersonal""",null,"""Rejected"""
"""8477430475517396483""","""7515253338893789882""",2.5231e18,"""BasicChecking""","""2023470075883199825""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'